In [ ]:
import keras
from keras.layers import *
from keras.models import *
from keras.losses import *
from keras.callbacks import *
from keras.optimizers import *
from keras import backend as K
from keras.datasets import mnist
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import os
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns

# Check GPU setup in Colab
%tensorflow_version 2.x 
import tensorflow as tf
# Silence the tensorflow warning message
tf.get_logger().setLevel('ERROR')

print("Tensorflow version: ", tf.__version__)
print(tf.test.gpu_device_name())
# Your expected output will be '/device:GPU:0'


Tensorflow version:  2.3.0
/device:GPU:0


## Load MNIST

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

image_size = x_train.shape[1]
# Only get some data to train and test
train_len = 60000
test_len = 10000
x_train = np.reshape(x_train[:train_len], [-1, image_size, image_size, 1])
x_test = np.reshape(x_test[:test_len], [-1, image_size, image_size, 1])
x_train = x_train.reshape(-1, 28, 28).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28).astype('float32') / 255
y_test = y_test[:test_len]
y_train = y_train[:train_len]

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# Change working directory to be current folder
# os.chdir('/content/gdrive/My Drive/Your Folder Name/Your sub Folder Name')
os.chdir('/content/gdrive/My Drive/Colab Notebooks/case_study_2')
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
case_study_2_discriminator.ipynb  vae_mnist.h5
case_study_2_vae.ipynb		  x_train_transformed.csv


In [ ]:
from keras.models import load_model

vae = load_model('vae_mnist.h5',compile=False)
vae

In [ ]:
x_train_transformed = vae.predict(x_train)
x_test_transformed = vae.predict(x_test)

In [ ]:
from numpy import savez_compressed

savez_compressed('x_train_transformed.npz', x_train_transformed)
savez_compressed('x_test_transformed.npz', x_test_transformed)

## Convolutional Neural Network Evaluator

In [ ]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

In [ ]:
def evaluator():
  model = Sequential()
  model.add(Conv2D(filters = 32, kernel_size = (4,4),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
  model.add(Conv2D(filters = 32, kernel_size = (4,4),padding = 'Same', 
                 activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
  model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(324, activation = "relu"))
  model.add(Dropout(0.3))
  model.add(Dense(10, activation = "softmax"))
  # Define the optimizer
  optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
  # Compile the model
  model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
  # Set a learning rate annealer
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
  return model

In [ ]:
!pip install livelossplot

In [ ]:
img_rows=x_train[0].shape[0]
img_cols=x_test[0].shape[1]

x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

x_train_transformed = x_train_transformed.reshape(x_train_transformed.shape[0],img_rows,img_cols,1)
x_test_transformed = x_test_transformed.reshape(x_test_transformed.shape[0],img_rows,img_cols,1)

In [ ]:
evaluator = evaluator()
from livelossplot import PlotLossesKeras

evaluator.fit(x_train_transformed, y_train, epochs=20,
            batch_size=10000,
            validation_data=(x_test, y_test),
          callbacks = [PlotLossesKeras()])

## Convolutional Neural Network Validator

In [ ]:
def validator():
  model = Sequential()
  model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
  model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2)))
  model.add(Dropout(0.2))
  model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
  model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(324, activation = "relu"))
  model.add(Dropout(0.1))
  model.add(Dense(10, activation = "softmax"))
  # Define the optimizer
  optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
  # Compile the model
  model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
  # Set a learning rate annealer
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
  return model

In [ ]:
validator = validator()

validator.fit(x_train, y_train, epochs=10,
            batch_size=10000,
            validation_data=(x_test_transformed, y_test),
          callbacks = [PlotLossesKeras()])

### Comparing Evaluator & Validator Results

In [ ]:
y_pred_v = validator.predict(x_test) # Predict encoded label as 2 => [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y_pred_v = np.argmax(y_pred_v, 1) # Decode Predicted labels
y_test = np.argmax(y_test, 1) # Decode labels

y_pred_e = evaluator.predict(x_test_transformed) # Predict encoded label as 2 => [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y_pred_e = np.argmax(y_pred_e, 1) # Decode Predicted labels

In [ ]:
evaluator_mat = confusion_matrix(y_test, y_pred_e) # Confusion matrix
fig = plt.figure(figsize=(10, 10))
# Plot Confusion matrix
plt.title('---------Confusion Matrix for Evaluator Predictions------------');
sns.heatmap(evaluator_mat.T, square=True, annot=True, cbar=False, cmap=plt.cm.Blues, fmt='.0f')
plt.xlabel('Predicted Values')
plt.ylabel('True Values');
plt.show();


validator_mat = confusion_matrix(y_test, y_pred_v) # Confusion matrix
fig = plt.figure(figsize=(10, 10))
# Plot Confusion matrix
plt.title('-------Confusion Matrix for Validator Predictions------------');
sns.heatmap(validator_mat.T, square=True, annot=True, cbar=False, cmap=plt.cm.Blues, fmt='.0f')
plt.xlabel('Predicted Values')
plt.ylabel('True Values');
plt.show();

In [ ]:
print('------------------ Evaluator Classification Report ----------------------')
print('       ')
print(classification_report(y_test, y_pred_e)) # Evaluator Classification report
print('------------------ Validator Classification Report ----------------------')
print('       ')
print(classification_report(y_test, y_pred_v)) # Validator Classification report